In [2]:
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

In [3]:
df = pd.read_csv('huh.csv')

Проведем label encoding

In [4]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for c in df.columns:
    df[c] = le.fit_transform(df[c])
df.head()

,Unnamed: 0,id,hour,day,month,source,destination,name,price,distance,...,icon,dewPoint,pressure,windBearing,cloudCover,uvIndex,ozone,moonPhase,precipIntensityMax,temperatureMin
0,0,165097,9,9,1,5,7,7,4,15,...,5,173,230,21,58,0,131,5,52,113
1,1,188776,2,13,0,5,7,2,20,15,...,6,258,98,40,82,0,69,9,53,118
2,2,378853,1,14,0,5,7,5,8,15,...,1,149,28,113,3,0,157,10,36,78
3,3,485300,4,16,0,5,7,4,53,15,...,1,61,158,163,0,0,69,12,0,67
4,4,558249,3,15,0,5,7,6,14,15,...,5,115,39,156,37,0,237,11,1,51


Для начала построим несколько бейслайнов для того, чтобы выбрать наилучший

In [5]:
X_to_split = df.drop(['price','surge_multiplier'], inplace=False, axis=1)
Y_to_split = df['price']

X_train,  X_test, Y_train, Y_test = train_test_split(X_to_split, Y_to_split, random_state=100)

dummy1 = DummyRegressor(strategy='mean')
dummy2 = DummyRegressor(strategy='median')

lr = LinearRegression()

dummy1.fit(X_train, Y_train)
dummy2.fit(X_train, Y_train)
lr.fit(X_train, Y_train)

LinearRegression()

Посмотрим, сколько выбросов в графе цена

In [11]:
print("Values larger than 100 " + str(Y_to_split[Y_to_split > 100].count()))
print("Values larger than 160 " + str(Y_to_split[Y_to_split > 160].count()))

Values larger than 100 2184
Values larger than 160 0


Так как явных выбросов (цена больше 160) в данных нет, а значения более 100 долларов составляют довольно малую часть, будем использовать метрику среднеквадратичной ошибки (MSE), которая является очень чувствительной (так как содержит возведение в квадрат).

In [13]:
from sklearn.metrics import mean_squared_error

y_preds_dummy1 = dummy1.predict(X_test)
print(f'MSE Dummy1 : {mean_squared_error(Y_test, y_preds_dummy1, squared=True)}')
y_preds_dummy2 = dummy2.predict(X_test)
print(f'MSE Dummy2 : {mean_squared_error(Y_test, y_preds_dummy2, squared=True)}')
y_preds_lr = lr.predict(X_test)
print(f'MSE Linear : {mean_squared_error(Y_test, y_preds_lr, squared=True)}')

MSE Dummy1 : 443.5180559696383
MSE Dummy2 : 473.85158689355086
MSE Linear : 232.16233744815963


Кажется, что, несмотря на то, что линейная регрессия предсказывает цены лучше, чем "глупые" регрессоры, ошибка все равно кажется большой. Попробуем в следующем задании как-то улучшить модель или использовать другие способы регрессии